# Import libraries

In [5]:
import flickrapi
from lxml import etree
from lxml.html.soupparser import fromstring
from lxml.etree import tostring
import bs4 as bs
import pandas as pd
from tqdm import tqdm
import time
from flickr_apikey import api_key_1, secret_1, api_key_2, secret_2, api_key_3, secret_3, api_key_4, secret_4

This function takes a list of tags and a list of boundary box and return csv files of photo ids, title, and location data in Singapore

# The function

In [8]:
def flickr(api_key_1, secret_1, api_key_2, secret_2, api_key_3, secret_3, api_key_4, secret_4,tags):
    for tag in tags:
        df_combined=pd.DataFrame()
        # for grid in tqdm(list_grid_combined,desc='inner_loop',position=0):
        page=1
        pbar = tqdm(total=16,desc='outer_loop',position=1, leave=False)

        while page <=16:
            # create a flickr object, search photos, and store the result
            if page%4==0:
                flickr = flickrapi.FlickrAPI(api_key_1,secret=secret_1)
                result=flickr.photos.search(api_key=api_key_1,
                                    tags=tag,
                                    bbox='103.6920359,1.1304753,104.0120359,1.4504753',
                                    accuracy=16, # the highest accruacy of location
                                    has_geo=1,
                                    geo_context=2, # outdoor photo
                                    content_type=1, # includes only photos
                                    per_page=250,
                                    page=page
                                    )
            elif page%4==1:
                flickr = flickrapi.FlickrAPI(api_key_2,secret=secret_2)
                result=flickr.photos.search(api_key=api_key_2,
                                    tags=tag,
                                    bbox='103.6920359,1.1304753,104.0120359,1.4504753',
                                    accuracy=16, # the highest accruacy of location
                                    has_geo=1,
                                    geo_context=2, # outdoor photo
                                    content_type=1, # includes only photos
                                    per_page=250,
                                    page=page
                                    )

            elif page%4==2:
                flickr = flickrapi.FlickrAPI(api_key_3,secret=secret_3)
                result=flickr.photos.search(api_key=api_key_3,
                                    tags=tag,
                                    bbox='103.6920359,1.1304753,104.0120359,1.4504753',
                                    accuracy=16, # the highest accruacy of location
                                    has_geo=1,
                                    geo_context=2, # outdoor photo
                                    content_type=1, # includes only photos
                                    per_page=250,
                                    page=page
                                    )

            else:
                flickr = flickrapi.FlickrAPI(api_key_4,secret=secret_4)
                result=flickr.photos.search(api_key=api_key_4,
                                    tags=tag,
                                    bbox='103.6920359,1.1304753,104.0120359,1.4504753',
                                    accuracy=16, # the highest accruacy of location
                                    has_geo=1,
                                    geo_context=2, # outdoor photo
                                    content_type=1, # includes only photos
                                    per_page=250,
                                    page=page
                                    )

            # parse the result to a string and convert it a soup object
            str_result=tostring(result, pretty_print=True).strip()
            soup=bs.BeautifulSoup(str_result,'lxml')
            photos=soup.find_all('photo') # find all the photos
            if len(photos)==0:
                break
            list_id=[]
            list_title=[]

            # store the id and title of each photo as lists
            for photo in photos:
                id=photo['id']
                title=photo['title']
                list_id.append(id)
                list_title.append(title)

            # convert lists to a dataframe
            dictionary_photo={'id': list_id, 'title': list_title}
            df_photos=pd.DataFrame.from_dict(dictionary_photo)

            # Getting location data
            list_lat=[]
            list_lon=[]
            api_condition=0
            for id in tqdm(list_id, desc='location',position=0, leave=True):
                if api_condition%4==0:
                    result_location=flickr.photos.geo.getLocation(api_key=api_key_1,photo_id=id)
                elif api_condition%4==1:
                    result_location=flickr.photos.geo.getLocation(api_key=api_key_2,photo_id=id)
                elif api_condition%4==2:
                    result_location=flickr.photos.geo.getLocation(api_key=api_key_3,photo_id=id)
                else:
                    result_location=flickr.photos.geo.getLocation(api_key=api_key_4,photo_id=id)
                str_result_location=tostring(result_location, pretty_print=True).strip()
                soup=bs.BeautifulSoup(str_result_location,'lxml')
                latitude=soup.find('location')['latitude']
                longitude=soup.find('location')['longitude']
                list_lat.append(latitude)
                list_lon.append(longitude)

                # update the condition
                api_condition+=1

            # add lists as columns to dataframe
            df_photos['latitude']=list_lat
            df_photos['longitude']=list_lon

            # append df_photos to df_combined
            df_combined=df_combined.append(df_photos, ignore_index=True)

            # update page number
            page+=1

            # update the progress bar  
            pbar.update(16)

        pbar.close()
        # save the result to a csv file
        df_combined.to_csv('singapore_'+tag+'.csv')

# Run the Function

In [7]:
if __name__ =='__main__':
    # tags
    tags=[
        'nature',
        'leisure',
        'waterfront',
        'relax',
        'chill',
        'history'
        ]

    # run the function
    flickr(api_key_1, secret_1, api_key_2, secret_2, api_key_3, secret_3, api_key_4, secret_4, tags)


outer_loop:   0%|          | 0/16 [00:00<?, ?it/s]

NameError: name 'api_key' is not defined